In [142]:
import io, time, json
import requests
from bs4 import BeautifulSoup
import urllib
from urllib import parse
import time
import pandas as pd

In [134]:
def retrieve_html(url, params = None):  
    if params is None:
        params = ''
    response = requests.get(url + urllib.parse.urlencode(params))
#     print(url + urllib.parse.urlencode(params))
    return response.status_code, response.text

In [135]:
def parse_page(html):
    soup = BeautifulSoup(html, 'html.parser')
    prd = soup.find("table", id="results")
    if prd is None:
        print('No results found')
        return
    prd = soup.find_all("tr")
    fund_val = prd[8].find_all("td")[-2].text
    return fund_val

In [136]:
def get_all_states(html):
    soup = BeautifulSoup(html, 'html.parser')
    states = soup.find("select", id="statefips").find_all("option")
    states_text = [t.text for t in states]
    states_values = [t['value'] for t in states]
    return states_values, states_text

In [137]:
def getFundingForQuery(url, params, search_html):
    state_list = get_all_states(search_html)[0]
    state_damage_vals = []
    for state in state_list:
        params['statefips'] = state
        state_damage_page = retrieve_html(url, params)[1]
        state_damage_vals.append(parse_page(state_damage_page))
    return state_damage_vals

In [138]:
def getFundingDataFromNOAA(eventType='(Z) Storm Surge/Tide', beginDate_mm='01', beginDate_dd='01', 
                           beginDate_yyyy='2017', endDate_mm='01', endDate_dd='31', endDate_yyyy='2018', 
                           county='ALL', statefips='48,TEXAS'):
    url = "https://www.ncdc.noaa.gov/stormevents/listevents.jsp?"
    params = {'eventType': eventType,
                  'beginDate_mm': beginDate_mm,
                  'beginDate_dd': beginDate_dd,
                  'beginDate_yyyy': beginDate_yyyy,
                  'endDate_mm': endDate_mm,
                  'endDate_dd': endDate_dd,
                  'endDate_yyyy': endDate_yyyy,
                  'county': county,
                  'sort': 'DT',
                  'submitbutton': 'Search',
                  'statefips': statefips}
    
    searchUrl = "https://www.ncdc.noaa.gov/stormevents/choosedates.jsp?statefips=-999%2CALL"
    search_html = retrieve_html(searchUrl)[1]
    damage_values = getFundingForQuery(url, params, search_html)
    return damage_values

In [139]:
# Get list of all county's corresponding to the states
def parse_county_page(html):
    soup = BeautifulSoup(html, 'html.parser')
    prd = soup.find('select', attrs={'name':'county'})
    if prd is None:
        print('No results found')
        return
    prd = prd.find_all("option")
    return prd

In [126]:
searchUrl = "https://www.ncdc.noaa.gov/stormevents/choosedates.jsp?"
search_html = retrieve_html(searchUrl)[1]
state_county = {}
for state in states_val:
    state_county[state] = []
    par = {'statefips':state}
    search_html = retrieve_html(searchUrl, params=par)[1]
    # print(search_html)
#     print('before', par)
    county_list = parse_county_page(search_html)
    if county_list is not None:
        state_county[state] = [i['value'] for i in county_list if i is not None]        
#     print('after',state_county[state])
#     break
# print(state_county)

No results found


In [143]:
#Now we have the dictionary of states and their countys. Lets extract all data for given state and county
def getFundingForCounty(url, params, search_html):
    state_list = get_all_states(search_html)[0]
    state_county_damage_val = {}
    for state in state_list:
        params['statefips'] = state
        state_county_damage_val[state] = {}
        for county in state_county[state]:
            params['county'] = county
            state_damage_page = retrieve_html(url, params)[1]
            state_county_damage_val[state][county] = parse_page(state_damage_page)
        print('State', state, ' for year ', params['beginDate_yyyy'])
        time.sleep(10)
        
    return state_county_damage_val

def getCountyLevelFundingDataFromNOAA(eventType='(Z) Storm Surge/Tide', beginDate_mm='01', beginDate_dd='01', 
                           beginDate_yyyy='2017', endDate_mm='01', endDate_dd='31', endDate_yyyy='2018', 
                           county='ALL', statefips='48,TEXAS'):
    url = "https://www.ncdc.noaa.gov/stormevents/listevents.jsp?"
    params = {'eventType': eventType,
                  'beginDate_mm': beginDate_mm,
                  'beginDate_dd': beginDate_dd,
                  'beginDate_yyyy': beginDate_yyyy,
                  'endDate_mm': endDate_mm,
                  'endDate_dd': endDate_dd,
                  'endDate_yyyy': endDate_yyyy,
                  'county': county,
                  'sort': 'DT',
                  'submitbutton': 'Search',
                  'statefips': statefips}
    
    searchUrl = "https://www.ncdc.noaa.gov/stormevents/choosedates.jsp?statefips=-999%2CALL"
    search_html = retrieve_html(searchUrl)[1]
    damage_values = getFundingForCounty(url, params, search_html)
    return damage_values

In [144]:
beginYearList = [2005, 2007, 2008, 2010, 2011, 2012, 2013, 2016, 2017]
year_state_county_damage = {}

for year in beginYearList:
    year_state_county_damage[year] = getCountyLevelFundingDataFromNOAA(beginDate_yyyy=str(year), endDate_yyyy=str(year))


State -999,ALL  for year  2005
State 1,ALABAMA  for year  2005
State 2,ALASKA  for year  2005
State 4,ARIZONA  for year  2005
State 5,ARKANSAS  for year  2005
State 6,CALIFORNIA  for year  2005
State 8,COLORADO  for year  2005
State 9,CONNECTICUT  for year  2005
State 10,DELAWARE  for year  2005
State 11,DISTRICT OF COLUMBIA  for year  2005
State 12,FLORIDA  for year  2005
State 13,GEORGIA  for year  2005
State 15,HAWAII  for year  2005
State 16,IDAHO  for year  2005
State 17,ILLINOIS  for year  2005
State 18,INDIANA  for year  2005
State 19,IOWA  for year  2005
State 20,KANSAS  for year  2005
State 21,KENTUCKY  for year  2005
State 22,LOUISIANA  for year  2005
State 23,MAINE  for year  2005
State 24,MARYLAND  for year  2005
State 25,MASSACHUSETTS  for year  2005
State 26,MICHIGAN  for year  2005
State 27,MINNESOTA  for year  2005
State 28,MISSISSIPPI  for year  2005
State 29,MISSOURI  for year  2005
State 30,MONTANA  for year  2005
State 31,NEBRASKA  for year  2005
State 32,NEVADA  fo

State 29,MISSOURI  for year  2010
State 30,MONTANA  for year  2010
State 31,NEBRASKA  for year  2010
State 32,NEVADA  for year  2010
State 33,NEW HAMPSHIRE  for year  2010
State 34,NEW JERSEY  for year  2010
State 35,NEW MEXICO  for year  2010
State 36,NEW YORK  for year  2010
State 37,NORTH CAROLINA  for year  2010
State 38,NORTH DAKOTA  for year  2010
State 39,OHIO  for year  2010
State 40,OKLAHOMA  for year  2010
State 41,OREGON  for year  2010
State 42,PENNSYLVANIA  for year  2010
State 44,RHODE ISLAND  for year  2010
State 45,SOUTH CAROLINA  for year  2010
State 46,SOUTH DAKOTA  for year  2010
State 47,TENNESSEE  for year  2010
State 48,TEXAS  for year  2010
State 49,UTAH  for year  2010
State 50,VERMONT  for year  2010
State 51,VIRGINIA  for year  2010
State 53,WASHINGTON  for year  2010
State 54,WEST VIRGINIA  for year  2010
State 55,WISCONSIN  for year  2010
State 56,WYOMING  for year  2010
State 81,LAKE ST CLAIR  for year  2010
State 84,HAWAII WATERS  for year  2010
State 85,G

State 56,WYOMING  for year  2013
State 81,LAKE ST CLAIR  for year  2013
State 84,HAWAII WATERS  for year  2013
State 85,GULF OF MEXICO  for year  2013
State 86,E PACIFIC  for year  2013
State 87,ATLANTIC SOUTH  for year  2013
State 88,ATLANTIC NORTH  for year  2013
State 89,GULF OF ALASKA  for year  2013
State 90,LAKE HURON  for year  2013
State 91,LAKE MICHIGAN  for year  2013
State 92,LAKE SUPERIOR  for year  2013
State 93,ST LAWRENCE R  for year  2013
State 94,LAKE ONTARIO  for year  2013
State 95,LAKE ERIE  for year  2013
State 96,VIRGIN ISLANDS  for year  2013
State 97,AMERICAN SAMOA  for year  2013
State 98,GUAM  for year  2013
State 99,PUERTO RICO  for year  2013
State -999,ALL  for year  2016
State 1,ALABAMA  for year  2016
State 2,ALASKA  for year  2016
State 4,ARIZONA  for year  2016
State 5,ARKANSAS  for year  2016
State 6,CALIFORNIA  for year  2016
State 8,COLORADO  for year  2016
State 9,CONNECTICUT  for year  2016
State 10,DELAWARE  for year  2016
State 11,DISTRICT OF COL

In [145]:
pd.DataFrame.from_dict(data=year_state_county_damage, orient='index').to_csv('State_County_Damage.csv', header=False)